# Lord of the Rings Spanner Graph Demo

## Initial setup

In [ ]:
!git clone https://github.com/cloudspannerecosystem/spanner-graph-notebook

In [ ]:
cd spanner-graph-notebook

In [ ]:
!pip install .

In [ ]:
%load_ext spanner_graphs

In [ ]:
!gcloud auth application-default login

## Spanner configuration

In [ ]:
PROJECT = "mh-test-lor-5" # @param
INSTANCE = "graph-demo" # @param
DATABASE = "lor_graph_db" # @param

## Person references graph

This graph draws direct relations for a given Character. Relation strength represent how frequent has been the interacion between characters. Incresing this value will show less nodes.

In [ ]:
persons = {'Aragorn':'arag', 'Arathorn':'arat', 'Arwen':'arwe', 'Balin':'bali', 'Beregond':'bere', 'Bilbo':'bilb', 'Bill':'bill', 'Boromir':'boro', 'Celeborn':'cele', 'Company':'comp', 'Denethor':'dene', 'Durin':'duri', 'Dwarves':'dwar', 'Elendil':'elen', 'Elrond':'elro', 'Elves':'elve', 'Ents':'ents', 'Éomer':'eome', 'Eorl':'eorl', 'Éowyn':'eowy', 'Faramir':'fara', 'Frodo':'frod', 'Galadriel':'gala', 'Gandalf':'ganda', 'Goldberry':'gber', 'Gildor':'gild', 'Gimli':'gimli', 'Glóin':'gloi', 'Glorfindel':'glorf', 'Gollum':'goll', 'Gorbag':'gorb', 'Wormtongue':'grim', 'Haldir':'hald', 'Hobbits':'hobb', 'Isildur':'isil', 'Legolas':'lego', 'Merry':'merr', 'Nine Riders':'nazg', 'Orcs':'orcs', 'Pippin':'pipp', 'Ring':'ring', 'Sam':'sams', 'Saruman':'saru', 'Sauron':'saur', 'Shadowfax':'sfax', 'Shelob':'shel', 'Théoden':'theod', 'Thorin':'thor', 'Thráin':'thra', 'Bombadil':'tomb', 'Treebeard':'treeb'}
places = {'Anduin':'andu', 'Bag End':'bage', 'Bree':'bree', 'Dark Tower':'dtow', 'Edoras':'edor', 'Gondor':'gond', 'Helm':'helm', 'Hobbiton':'hton', 'Isengard':'isen', 'Lórien':'lori', 'Lothlórien':'loth', 'Mount Doom':'mdoo', 'Mirkwood':'mirk', 'Mordor':'mord', 'Morgul':'morg', 'Moria':'mori', 'Númenor':'nume', 'Old Forest':'oldf', 'Orthanc':'orth', 'Osgiliath':'osgi', 'Rivendell':'rive', 'Rohan':'roha', 'Shire':'shir', 'Minas Tirith':'tiri'}

In [ ]:
PERSON = "Aragorn" #@param ['Aragorn', 'Arathorn', 'Arwen', 'Balin', 'Beregond', 'Bilbo', 'Bill', 'Boromir', 'Celeborn', 'Company', 'Denethor', 'Durin', 'Dwarves', 'Elendil', 'Elrond', 'Elves', 'Ents', 'Éomer', 'Eorl', 'Éowyn', 'Faramir', 'Frodo', 'Galadriel', 'Gandalf', 'Goldberry', 'Gildor', 'Gimli', 'Glóin', 'Glorfindel', 'Gollum', 'Gorbag', 'Wormtongue', 'Haldir', 'Hobbits', 'Isildur', 'Legolas', 'Merry', 'Nine Riders', 'Orcs', 'Pippin', 'Ring', 'Sam', 'Saruman', 'Sauron', 'Shadowfax', 'Shelob', 'Théoden', 'Thorin', 'Thráin', 'Bombadil', 'Treebeard']
RELATION_STRENGTH = 9 # @param {type:"slider", min:1, max:533, step:1}


In [ ]:
query = f'''
GRAPH LoRGraph
MATCH query_path = (p1:Persons)-[ref:Reference]->(p2:Persons)
WHERE ref.times > {RELATION_STRENGTH}
AND p1.id='{persons[PERSON]}' OR p2.id='{persons[PERSON]}'
RETURN TO_JSON(query_path) AS path_json
'''

print(query)

Copy the query above and paste into the cell below:


In [ ]:
%%spanner_graph --project {PROJECT} --instance {INSTANCE} --database {DATABASE}

GRAPH LoRGraph
MATCH query_path = (p1:Persons)-[ref:Reference]->(p2:Persons)
WHERE ref.times > 9
AND p1.id='arag' OR p2.id='arag'
RETURN TO_JSON(query_path) AS path_json


## Persons and places graph

This graph adds places to the query, so each character is required to have been in one of those places.

In [ ]:
Mordor = True # @param {type:"boolean"}
Helm = True # @param {type:"boolean"}
Mount_Doom = True # @param {type:"boolean"}
Rivendell  = True # @param {type:"boolean"}
Rohan  = True # @param {type:"boolean"}
Minas_Tirith  = True # @param {type:"boolean"}
Dark_Tower = True # @param {type:"boolean"}
Isengard = True # @param {type:"boolean"}

places_list =''
if(Mordor) :
  places_list += '\'' + places['Mordor'] + '\','
if(Helm) :
  places_list += '\'' + places['Helm'] + '\','
if(Mount_Doom) :
  places_list += '\'' + places['Mount Doom'] + '\','
if(Rivendell) :
  places_list += '\'' + places['Rivendell'] + '\','
if(Rohan) :
  places_list += '\'' + places['Rohan'] + '\','
if(Minas_Tirith) :
  places_list += '\'' + places['Minas Tirith'] + '\','
if(Dark_Tower) :
  places_list += '\'' + places['Dark Tower'] + '\','
if(Isengard) :
  places_list += '\'' + places['Isengard'] + '\','
places_list = places_list[:len(places_list)-1]


In [ ]:
query = f'''GRAPH LoRGraph
MATCH query_path = (p1:Persons)-[ref:Reference]->(p2:Persons)
WHERE ref.times > {RELATION_STRENGTH}
AND p1.id='{persons[PERSON]}' OR p2.id='{persons[PERSON]}'
RETURN p1, p2, ref
NEXT
MATCH (p1:Persons)-[placeRef1:PlacesPersons]->(place:Places)
MATCH (p2:Persons)-[placeRef2:PlacesPersons]->(place:Places)
WHERE place.id IN ({places_list})
RETURN TO_JSON(p1) AS p1, TO_JSON(p2) AS p2, TO_JSON(place) AS place, TO_JSON(ref) AS ref, TO_JSON(placeRef1) AS placeRef1, TO_JSON(placeRef2) AS placeRef2
'''

print(query)

Copy the query above and paste into the cell below:

In [ ]:
%%spanner_graph --project {PROJECT} --instance {INSTANCE} --database {DATABASE}

GRAPH LoRGraph
MATCH query_path = (p1:Persons)-[ref:Reference]->(p2:Persons)
WHERE ref.times > 9
AND p1.id='arag' OR p2.id='arag'
RETURN p1, p2, ref
NEXT
MATCH (p1:Persons)-[placeRef1:PlacesPersons]->(place:Places)
MATCH (p2:Persons)-[placeRef2:PlacesPersons]->(place:Places)
WHERE place.id IN ('mord','helm','mdoo','rive','roha','tiri','dtow','isen')
RETURN TO_JSON(p1) AS p1, TO_JSON(p2) AS p2, TO_JSON(place) AS place, TO_JSON(ref) AS ref, TO_JSON(placeRef1) AS placeRef1, TO_JSON(placeRef2) AS placeRef2
